In [1]:
import pickle
import matplotlib.pyplot as plt

plt.rcParams.update(plt.rcParamsDefault)
import numpy as np
import mat73
import scipy.io
import h5py
import networkx as nx
import networkx as nx
import pygenstability as pgs

be_data = scipy.io.loadmat('/Users/sonmjack/Downloads/simon_paper/data_fam1novfam1_trackdata.mat')
mat_trigger = np.load('/Users/sonmjack/Downloads/simon_paper/shengyuan_trigger_fam1.npy')
type_array = h5py.File('/Users/sonmjack/Downloads/simon_paper/data_fam1novfam1_timeseries.mat')

gene = type_array['genotype'][:, :].T
# mat_label = np.zeros((gene.shape[0],4))
# be_phi_sum = be_data['nov_phi']
be_phi_sum_fam1 = be_data['fam1_phi']
be_phi_sum_nov = be_data['nov_phi']
be_phi_sum_fam1r2 = be_data['fam1r2_phi']
be_x = be_data['fam1_x']
be_y = be_data['fam1_y']
be_time = be_data['fam1_time']
be_speed = be_data['fam1_speed']
include_list = []
be_x_list_old = []
be_y_list_old  = []
be_time_list_old  = []
be_speed_list_old  = []
be_phi_list_old_fam1 = []
be_phi_list_old_nov = []
be_phi_list_old_fam1r2 = []
gene_list = []
for i in range(0,10,2):#0, len(mat_trigger), 2

    if i == len(mat_trigger):
        break
    if i == 18:
        pass
    else:
        be_x_list_old.append(be_x[int(i/2),0])
        be_y_list_old.append(be_y[int(i / 2), 0])
        be_time_list_old.append(be_time[int(i / 2), 0])
        be_speed_list_old.append(be_speed[int(i / 2), 0])
        be_phi_list_old_fam1.append(be_phi_sum_fam1[int(i / 2), 0])
        be_phi_list_old_nov.append(be_phi_sum_nov[int(i / 2), 0])
        be_phi_list_old_fam1r2.append(be_phi_sum_fam1r2[int(i / 2), 0])
        gene_list.append(mat_trigger[i, 1])
del be_x, be_data,  be_y, be_time, be_speed

with open('/Users/sonmjack/Downloads/age10 result_fam1/fam1_all_Smask.pkl', 'rb') as file:
    mask = pickle.load(file)

env = 'nov'
#env = 'fam1'
#env = 'fam1r2'
if env == 'fam1':
    with open('/Users/sonmjack/Downloads/age10 result_fam1/fam1_all_EPSP.pkl', 'rb') as file:
        dy_list = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age10 result_fam1/fam1_S_spike.pkl', 'rb') as file:
        neuron_spike = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age10 result_fam1/fam1_signal_corr_AD', 'rb') as file:
        signal_list = pickle.load(file)
elif env == "nov":
    with open('/Users/sonmjack/Downloads/age10 result_nov/nov_all_EPSP.pkl', 'rb') as file:
        dy_list = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age10 result_nov/nov_S_spike.pkl', 'rb') as file:
        neuron_spike = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age10 result_nov/nov_signal_corr_AD', 'rb') as file:
        signal_list = pickle.load(file)
elif env == 'fam1r2':
    with open('/Users/sonmjack/Downloads/age10 result_fam1r2/fam1r2_all_EPSP.pkl', 'rb') as file:
        dy_list = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age10 result_fam1r2/fam1r2_S_spike.pkl', 'rb') as file:
        neuron_spike = pickle.load(file)
    with open('/Users/sonmjack/Downloads/age10 result_fam1r2/fam1r2_signal_corr_AD', 'rb') as file:
        signal_list = pickle.load(file)


In [ ]:
# import pandas as pd
# import seaborn as sns
# for i in range(len(be_phi_list_old_fam1)):
#     if gene_list[i] == 116:
#         angle_fam1 = be_phi_list_old_fam1[i].reshape(-1)
#         angle_nov = be_phi_list_old_nov[i].reshape(-1)
#         angle_fam1r2 = be_phi_list_old_fam1r2[i].reshape(-1)
#         bins = np.arange(0, 370, 10)  # 创建从0到360度，步长为10的区间
#         angle_categories = pd.cut(angle_nov, bins, right=False, labels=[f'{i}°-{i+10}°' for i in range(0, 360, 10)])
#         
#         # 计算每个区间的占比
#         angle_distribution = pd.value_counts(angle_categories, normalize=True).sort_index()
#         
#         # 将结果转换为DataFrame以便使用Seaborn进行绘图
#         df = pd.DataFrame({'Angle Range': angle_distribution.index, 'Percentage': angle_distribution.values})
#         
#         # 绘制条形图
#         plt.figure(figsize=(14, 7))
#         sns.barplot(x='Angle Range', y='Percentage', data=df)
#         plt.xticks(rotation=90)  # 旋转x轴标签以便更容易阅读
#         plt.title('Distribution of Angles of'+str(i))
#         plt.ylabel('Percentage')
#         plt.xlabel('Angle Interval')
#         plt.show()
#         
# velocity



In [ ]:

def Connector(Q):
    D = nx.to_networkx_graph(Q, create_using=nx.DiGraph())
    Isolate_list = list(nx.isolates(D))
    if len(Isolate_list) > 0:
        for i in Isolate_list:
            if i == 0:
                Q[i + 1, i] = 0.0001
            else:
                Q[i - 1, i] = 0.0001
    del D
    return Q
def normal(A):
    np.fill_diagonal(A, 0)
    min_val = np.min(A)
    max_val = np.max(A)
    A = (A - min_val) / (max_val - min_val)

    return A

def sparse (A):
    N = A.shape[0]
    # print(max(A[:,4]))
    # A=np.where(A > 0.09, 1, 0)
    k = int(N/2)

    # W.sort(reverse=True)
    B1 = np.zeros((N, N))
    for i in range(N):
        W = sorted(A[i, :], reverse=True)
        #     print( W[k])
        B1[i, :] = np.where(A[i, :] > W[k], 1, 0)

    # B=np.multiply(B1,A)
    # print(W[k])
    # print(A[20,1:20])
    # print(B[20,1:20])

    C1 = np.zeros((N, N))


    for i in range(N):
        W = sorted(A[:, i], reverse=True)
        #     print( W[k])
        C1[:, i] = np.where(A[:, i] > W[k], 1, 0)
    # C=np.multiply(C1,A)
    Q1 = B1 + C1
    Q2 = np.where(Q1 > .9, 1, 0)

    Q = np.multiply(Q2, A)


    # del A
    for i in range(Q.shape[0]):
        # 检查该行是否全为零
        if np.all(Q[i] == 0):
            # 如果是全为零，随机选择一个元素，并将其赋值为 0.001
            #random_index = np.random.randint(0, Q.shape[1]-1)-1  # 随机选择一个列索引
            Q[i, i-1] = 0.001
            Q[i-1,i] = 0.001
    return Q

import plotly.graph_objects as go  # pragma: no cover
from plotly.offline import plot

def plot_sankey(
    all_results,
    optimal_scales=True,
    live=False,
    filename='',
    scale_index=None,
):  # pragma: no cover
    """Plot Sankey diagram of communities accros scale (plotly only).
    Args:
        all_results (dict): results from run function
        optimal_scales (bool): use optimal scales or not
        live (bool): if True, interactive figure will appear in browser
        filename (str): filename to save the plot
        scale_index (bool): plot scale of indices
    """
    sources = []
    targets = []
    values = []
    shift = 0

    # if not scale_index:
    all_results["community_id_reduced"] = all_results["community_id"][0::10]

    community_ids = all_results["community_id_reduced"]

    for i in range(len(community_ids) - 1):
        community_source = np.array(community_ids[i])
        community_target = np.array(community_ids[i + 1])
        source_ids = set(community_source)
        target_ids = set(community_target)
#         print(target_ids)
        for source in source_ids:
            for target in target_ids:
                value = sum(community_target[community_source == source] == target)
#                 print(community_target[community_source == source] == target)
                if value > 0:
                    values.append(value)
                    sources.append(source + shift)
                    targets.append(target + len(source_ids) + shift)
        shift += len(source_ids)

    fig = go.Figure(
        data=[
            go.Sankey(
                node={
                    "pad": 1,
                    "thickness": 1,
                    "line": {"color": "black", "width": 0.0},
                },
                link={"source": sources, "target": targets, "value": values},
            )
        ],
    )

    fig.update_layout(showlegend=False)
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    fig.update_layout(
    paper_bgcolor='white',  # 设置图表的背景颜色为白色
    plot_bgcolor='white')
    
    Scale_enumerate=[r'$S_{%s}$' % (len(community_ids)-i)  for i in range(len(community_ids))]
    for x_coordinate, column_name in enumerate(Scale_enumerate):
        fig.add_annotation(
              x=x_coordinate,#Plotly recognizes 0-5 to be the x range.

              y=1.075,#y value above 1 means above all nodes
              xref="x",
              yref="paper",
              text=column_name,#Text
              showarrow=False,
              font=dict(
                  family="Tahoma",
                  size=16,
                  color="black"
                  ),
              align="left",
              )
    fig.write_image(filename)

In [ ]:
for index in range(len(signal_list)):
    if gene_list[index] == 119 or gene_list[index] == 116:
        type = 'AD'
        Signal = signal_list[index]
        Signal[Signal <= 0] = 0
        Signal[np.isinf(Signal)] = 0
        Signal = np.nan_to_num(Signal, nan=0)

        Q = sparse(Signal)
        D = nx.to_networkx_graph(Q)
        adjacency = nx.adjacency_matrix(D)

        all_results = pgs.run(adjacency,
                              min_scale=-1.5,
                              max_scale=1.5,
                              n_scale=200,
                              n_tries=600,
                              constructor="continuous_normalized")

        if env == 'fam1':
            with open('/Users/sonmjack/Downloads/age10 result_fam1/fam1_Signal_Markov_' + str(index) + '.pkl',
                      'wb') as f:
                pickle.dump(all_results, f)
        elif env == "nov":
            with open('/Users/sonmjack/Downloads/age10 result_nov/nov_Signal_Markov_' + str(index) + '.pkl', 'wb') as f:
                pickle.dump(all_results, f)
        elif env == 'fam1r2':
            with open('/Users/sonmjack/Downloads/age10 result_fam1r2/fam1r2_Signal_Markov_' + str(index) + '.pkl',
                      'wb') as f:
                pickle.dump(all_results, f)
    else:
        pass

In [10]:
import numpy as np
import matplotlib.colors as mcolors
import random

Color_Code2 = ['floralwhite', 'lightskyblue', "red", "orange", "pink", "olive", "cyan", "black", "yellow", "green",
               "brown", "gray", "tomato"]

list_color = []
for color in mcolors.CSS4_COLORS:
    list_color.append(color)
random.shuffle(list_color)
Color_Code1 = [*Color_Code2, *list_color]
Color_Code = [*Color_Code1, *list_color]

z = 0
for index in range(len(dy_list)):  #len(dy_list)
    if gene_list[index] == 116:
        # Spike_train = neuron_spike[index]
        # mask_index = mask[index]
        Spike_train = neuron_spike[index]
        mask_index = mask[index]
        type = 'wild type'
        if env == 'fam1':
            with open('/Users/sonmjack/Downloads/age10 result_fam1/fam1_Signal_Markov_' + str(z) + '.pkl', 'rb') as file:
                all_results = pickle.load(file)
            _ = pgs.plot_scan(all_results)
            plt.savefig(
                '/Users/sonmjack/Downloads/age10 result_fam1/Signal_Markov_fig/' + str(index) + '_old_fam1_markov.png')
            plt.close()

        elif env == "nov":
            with open('/Users/sonmjack/Downloads/age10 result_nov/nov_Signal_Markov_' + str(z) + '.pkl', 'rb') as file:
                all_results = pickle.load(file)
            _ = pgs.plot_scan(all_results)
            plt.savefig(
                '/Users/sonmjack/Downloads/age10 result_nov/Signal_Markov_fig/' + str(index) + '_old_nov_markov.png')
            plt.close()
        elif env == "fam1r2":
            with open('/Users/sonmjack/Downloads/age10 result_fam1r2/fam1r2_Signal_Markov_' + str(z) + '.pkl',
                      'rb') as file:
                all_results = pickle.load(file)
            _ = pgs.plot_scan(all_results)
            plt.savefig('/Users/sonmjack/Downloads/age10 result_fam1r2/Signal_Markov_fig/' + str(index) + '_old_fam1r2_markov.png')
            plt.close()
            
        # if gene_list[index] == 119:
        #     Spike_train = neuron_spike[index]
        #     mask_index = mask[index]
        #     type = 'wild type'
        #     if env == 'fam1':
        #         with open('/Users/sonmjack/Downloads/age10 result_fam1/fam1_S_Markov_'+ str(index) +'.pkl', 'rb') as file:
        #             all_results = pickle.load(file)
        #         _ = pgs.plot_scan(all_results)
        #         plt.savefig('/Users/sonmjack/Downloads/age10 result_fam1/Markov_fig/'+str(index)+'_young_fam1_markov.png')
        #         plt.close()
        #         
        #     elif env =="nov":
        #         with open('/Users/sonmjack/Downloads/age10 result_nov/nov_S_Markov_'+ str(index) +'.pkl', 'rb') as file:
        #             all_results = pickle.load(file)
        #         _ = pgs.plot_scan(all_results)
        #         plt.savefig('/Users/sonmjack/Downloads/age10 result_nov/Markov_fig/'+str(index)+'_young_nov_markov.png')
        #         plt.close()

        if env == 'fam1' or env == 'fam1r2':
            t_opt = 1
            selected_partitions = all_results['selected_partitions']
            Community = all_results['community_id'][selected_partitions[-t_opt]]
        elif env == "nov":
            selected_partitions = all_results['selected_partitions']
            Community = all_results['community_id'][199]

        non_zero_indices_per_row = []
        for row in Spike_train:
            # 找到每行中不为0的元素的列索引
            non_zero_indices = np.where(row != 0)[0]
            # 添加到列表中
            non_zero_indices_per_row.append(list(non_zero_indices))
        neuralData = []
        ColorCode = []

        it = 0
        Nodes = range(len(non_zero_indices_per_row))
        for m, j in zip(Nodes, Nodes):
            for i in range(max(Community) + 1):
                if Community[j] == i:
                    non_zero_indices_per_row[int(m)].sort()
                    neuralData.append(non_zero_indices_per_row[int(m)])
                    ColorCode.append(Color_Code[i + 1])
                    # A_ordered_row[it,:]=Q[j,:]
                    it += 1
        # it=0
        # for i in range(max(Community)+1):
        #     for m,j in zip(Nodes,range(N)):
        #         if Community[j]==i:
        #             A_ordered[:,it]=A_ordered_row[:,j]
        #             it+=1
        plt.eventplot(neuralData, color=ColorCode)
        plt.gca().invert_yaxis()
        # plot.xlim([8000,9000])
        # plot.title('Spike raster plot')
        # plt.title(r"%s,  $N_g=%d$" % (data[l],max(Community)+1), fontweight ='bold',)
        # plt.title(r"%s,  $S_%d$,  $N_c=%d$" % (data[l],t_opt,max(Community)+1),fontsize = 22)
        plt.title("Reordered neural spike trains " + f'N-{max(Community) + 1}')
        plt.xlabel('Time', fontsize=13)
        plt.ylabel('Rearranged Neuron ID', fontsize=13)
        plt.yticks(fontsize=13)

        max_in_sublists = [max(sublist) for sublist in neuralData if sublist]

        # 找到最大值中的最大值
        overall_max = max(max_in_sublists)
        nearest_5000_multiple = round(overall_max / 10000) * 10000
        plt.xticks(np.arange(0, nearest_5000_multiple, 10000),
                   [str(int(i * 10000 / 30)) for i in range(0, round(overall_max / 10000))], fontsize=13)
        #plt.show()

        if env == 'fam1':
            plt.savefig('/Users/sonmjack/Downloads/age10 result_fam1/Signal_Markov_fig/neuron_list_' + str(
                index) + '_young_fam1_markov.png')
            plt.close()

        elif env == "nov":
            plt.savefig('/Users/sonmjack/Downloads/age10 result_nov/Signal_Markov_fig/neuron_list_' + str(
                index) + '_young_nov_markov.png')
            plt.close()
        elif env == "fam1r2":
            plt.savefig('/Users/sonmjack/Downloads/age10 result_fam1r2/Signal_Markov_fig/neuron_list_' + str(
                index) + '_young_fam1r2_markov.png')
            plt.close()

        from matplotlib.colors import BoundaryNorm

        pic = np.zeros((512, 512))
        for i in range(mask_index.shape[2]):
            index_row = np.where(mask_index[:, :, i] == True)[0]
            index_con = np.where(mask_index[:, :, i] == True)[1]
            for j in range(len(index_row)):
                pic[index_row[j], index_con[j]] = Community[i] + 1
        #cmap = sns.diverging_palette(220, 20, as_cmap=True)
        import seaborn as sns

        cmap = plt.get_cmap('tab10')

        from matplotlib.colors import LinearSegmentedColormap

        cmap = LinearSegmentedColormap.from_list("custom_cmap", Color_Code2, N=len(Color_Code2))
        bounds = np.linspace(-0.5, 11.5, 13)
        norm = BoundaryNorm(bounds, cmap.N)
        sns.heatmap(pic, cmap=cmap, norm=norm)

        plt.title('Community beasd on real data')
        #plt.show()
        if env == 'fam1':
            plt.savefig('/Users/sonmjack/Downloads/age10 result_fam1/Signal_Markov_fig/mask_' + str(
                index) + '_young_fam1_markov_neuron.png')
            plt.close()

        elif env == "nov":
            plt.savefig('/Users/sonmjack/Downloads/age10 result_nov/Signal_Markov_fig/mask_' + str(
                index) + '_young_nov_markov_neuron.png')
            plt.close()
        elif env == "fam1r2":
            plt.savefig('/Users/sonmjack/Downloads/age10 result_fam1r2/Signal_Markov_fig/mask_' + str(
                index) + '_young_fam1r2_markov_neuron.png')
            plt.close()

        import plotly.graph_objects as go  # pragma: no cover
        from plotly.offline import plot  # pragma: no cover
        import sys


        def plot_sankey(
                all_results,
                optimal_scales=True,
                live=False,
                filename='',
                scale_index=None,
        ):  # pragma: no cover
            """Plot Sankey diagram of communities accros scale (plotly only).
            Args:
                all_results (dict): results from run function
                optimal_scales (bool): use optimal scales or not
                live (bool): if True, interactive figure will appear in browser
                filename (str): filename to save the plot
                scale_index (bool): plot scale of indices
            """
            sources = []
            targets = []
            values = []
            shift = 0

            # if not scale_index:
            all_results["community_id_reduced"] = all_results["community_id"][0::10]
            # else:
            #     all_results["community_id_reduced"] = [all_results["community_id"][i] for i in scale_index]

            community_ids = all_results["community_id_reduced"]
            # if optimal_scales and ("selected_partitions" in all_results.keys()):
            #     community_ids = [community_ids[u] for u in all_results["selected_partitions"]]

            # Community=all_results['community_id'][selected_partitions[-t_opt]]

            for i in range(len(community_ids) - 1):
                community_source = np.array(community_ids[i])
                community_target = np.array(community_ids[i + 1])
                source_ids = set(community_source)
                target_ids = set(community_target)
                #         print(target_ids)
                for source in source_ids:
                    for target in target_ids:
                        value = sum(community_target[community_source == source] == target)
                        #                 print(community_target[community_source == source] == target)
                        if value > 0:
                            values.append(value)
                            sources.append(source + shift)
                            targets.append(target + len(source_ids) + shift)
                shift += len(source_ids)

            fig = go.Figure(
                data=[
                    go.Sankey(
                        node={
                            "pad": 1,
                            "thickness": 1,
                            "line": {"color": "black", "width": 0.0},
                        },
                        link={"source": sources, "target": targets, "value": values},
                    )
                ],
            )

            fig.update_layout(showlegend=False)
            fig.update_xaxes(visible=False)
            fig.update_yaxes(visible=False)
            fig.update_layout(
                paper_bgcolor='white',  # 设置图表的背景颜色为白色
                plot_bgcolor='white')

            #
            #     fig.update_layout(
            #         title=data,
            # #         xaxis_title="X Axis Title",
            # #         yaxis_title="Y Axis Title",
            # #         legend_title="Legend Title",
            # #         font=dict(
            # #             family="Courier New, monospace",
            # #             size=18,
            # #             color="RebeccaPurple")
            #     )

            Scale_enumerate = [r'$S_{%s}$' % (len(community_ids) - i) for i in range(len(community_ids))]
            for x_coordinate, column_name in enumerate(Scale_enumerate):
                fig.add_annotation(
                    x=x_coordinate,  #Plotly recognizes 0-5 to be the x range.

                    y=1.075,  #y value above 1 means above all nodes
                    xref="x",
                    yref="paper",
                    text=column_name,  #Text
                    showarrow=False,
                    font=dict(
                        family="Tahoma",
                        size=16,
                        color="black"
                    ),
                    align="left",
                )
            fig.write_image(filename)
            #plot(fig, filename=filename)


        #     if live:
        #         fig.show()

        #     return fig

        if env == 'fam1':
            name = '/Users/sonmjack/Downloads/age10 result_fam1/Signal_Markov_fig/snakey' + str(
                index) + '_young_fam1.png'
        elif env == "nov":
            name = '/Users/sonmjack/Downloads/age10 result_nov/Signal_Markov_fig/snakey' + str(index) + '_young_nov.png'
        elif env == "fam1r2":
            name = '/Users/sonmjack/Downloads/age10 result_fam1r2/Signal_Markov_fig/snakey' + str(
                index) + '_young_fam1r2.png'

        plot_sankey(
            all_results,
            optimal_scales=True,
            live=False,
            filename=name,
            scale_index=None,
        )
        z = z + 1
